In [1]:
import torch
import ksc.torch_frontend as knossos


In [19]:
def tsrelu2(x : torch.Tensor) -> torch.Tensor:
    return x*x*17

v = torch.randn(2, device='cuda')

tsrelu2(v)

ts = torch.jit.script(tsrelu2)

print("First graph - with profiling annotations")
print(ts.graph_for(v))


print("Steady state, without grad")
print(ts.graph_for(v))


v.requires_grad = True

ts = torch.jit.script(tsrelu2)

print("Steady state, with grad")
print(ts.graph_for(v))



First graph - with profiling annotations
graph(%x.1 : Tensor):
  %1 : int = prim::Constant[value=17]() # /tmp/ipykernel_66434/3900783400.py:2:15
  %4 : Tensor = prim::profile[profiled_type=Float(2, strides=[1], requires_grad=1, device=cuda:0)](%x.1)
  %5 : Tensor = prim::profile[profiled_type=Float(2, strides=[1], requires_grad=1, device=cuda:0)](%x.1)
  %2 : Tensor = aten::mul(%4, %5) # /tmp/ipykernel_66434/3900783400.py:2:11
  %6 : Tensor = prim::profile[profiled_type=Float(2, strides=[1], requires_grad=1, device=cuda:0)](%2)
  %3 : Tensor = aten::mul(%6, %1) # /tmp/ipykernel_66434/3900783400.py:2:11
  %7 : Tensor = prim::profile[profiled_type=Float(2, strides=[1], requires_grad=1, device=cuda:0)](%3)
   = prim::profile()
  return (%7)

Steady state, without grad
graph(%x.1 : Float(2, strides=[1], requires_grad=1, device=cuda:0)):
  %18 : Float(2, strides=[1], requires_grad=1, device=cuda:0), %19 : bool = prim::RequiresGradCheck[types=[Tensor(requires_grad=1)]](%x.1)
  %20 : Tensor =

In [2]:
@knossos.register(generate_lm=True)
def relu2(x : float) -> float:
  if x < 0.0:
    return 0.0
  else:
    return x*x

relu2.ensure_compiled((1.0,))

relu2(3.3)

Module=None
knossos.register: Compiling relu2
knossos: module= None
knossos: no module, just returning None
TorchScriptVisitor: Adding <function relu2 at 0x7ff260a179d0> to todo set()
TorchScriptVisitor: Remaining: {<function relu2 at 0x7ff260a179d0>}
TorchScriptVisitor: Marking <function relu2 at 0x7ff260a179d0> as done
(def relu2 None ((_x$o1 : Float))
  (let (_2 0.0)
  (let (_3 (aten::lt _x$o1 0.0))
  (let (_16 (if _3
              _2
              (let (_7 (aten::mul _x$o1 _x$o1))
              _7)))
  _16))))

generate_cpp_from_ks: /awf-vmdisk-1/awf/dev/knossos-ksc/build/bin/ksc /tmp/tmpck7qbt3e.ks
ksc: read decls
ksc: Linted generated defs
ksc: Linted opt decls
ksc: Linted CSE decls
ksc: Writing to /tmp/tmprkx0uy80.kso
ksc: Writing to /tmp/tmpat2335sy.cpp


ksc.utils: File not changed: /awf-vmdisk-1/awf/dev/knossos-ksc/build/torch_extensions/KscStub_VSnone_lm_NoMod_relu2__generate_lm/ksc-main.cpp
ksc.utils: File not changed: /awf-vmdisk-1/awf/dev/knossos-ksc/build/torch_extension

tensor(10.8900)

In [3]:
vrelu2 = knossos.vmap(relu2)

Module=None


In [4]:
x = torch.rand(1000000)
y = vrelu2(x)
dy = y * 0.0001
vrelu2.vjp(x, dy)

knossos.register: Compiling relu2
knossos: module= None
knossos: no module, just returning None
TorchScriptVisitor: Adding <function relu2 at 0x7ff260a179d0> to todo set()
TorchScriptVisitor: Remaining: {<function relu2 at 0x7ff260a179d0>}
TorchScriptVisitor: Marking <function relu2 at 0x7ff260a179d0> as done
TorchScriptVisitor: example input type (Tensor 0 Float) differs from TorchScript input type Float
(def relu2 None ((_x$o1 : Float))
  (let (_2 0.0)
  (let (_3 (aten::lt _x$o1 0.0))
  (let (_16 (if _3
              _2
              (let (_7 (aten::mul _x$o1 _x$o1))
              _7)))
  _16))))

generate_cpp_from_ks: /awf-vmdisk-1/awf/dev/knossos-ksc/build/bin/ksc /tmp/tmpeil5jnx9.ks
ksc: read decls
ksc: Linted generated defs
ksc: Linted opt decls
ksc: Linted CSE decls
ksc: Writing to /tmp/tmpca5jqi15.kso
ksc: Writing to /tmp/tmpiu234asy.cpp




torch::Tensor entry(torch::Tensor arg0)
     {
    int64_t n = arg0.size(0);

    KS_ASSERT(arg0.is_contiguous());
    KS_ASSERT(arg0.scal

tensor([2.2486e-05, 1.9764e-04, 1.9334e-07,  ..., 1.7217e-04, 2.6240e-08,
        6.6577e-05])

In [5]:
a = torch.randn(4,3)
sum(a[i,j] for i in range(4) for j in range(3))

tensor(6.0630)

In [6]:
@knossos.register(generate_lm=True)
def conv2(a : torch.Tensor, b : torch.Tensor) -> torch.Tensor:
  ma,na = a.size()
  mb,nb = b.size()
  out = torch.empty(ma-mb+1,na-nb+1)
  for ia in range(ma-mb+1):
    for ja in range(na-nb+1):
      out[ia, ja] = sum(torch.tensor([
          sum(torch.tensor([
            a[ia+ib,ja+jb] * b[ib,jb]
            for ib in range(ma)
          ]))
          for jb in range(na)
      ]))
  return out 

a = torch.randn(128,128)
b = torch.randn(3,3)

conv2(a,b)


Module=None
knossos.register: Compiling conv2
knossos: module= None
knossos: no module, just returning None
TorchScriptVisitor: Adding <function conv2 at 0x7ff260a17ee0> to todo set()
TorchScriptVisitor: Remaining: {<function conv2 at 0x7ff260a17ee0>}
TorchScriptVisitor: Marking <function conv2 at 0x7ff260a17ee0> as done
 = prim::Loop(%30, %33) # /tmp/ipykernel_66434/1243303194.py:6:2
  block0(%ia.1 : int):
    %37 : int = aten::sub(%na.1, %nb.1) # /tmp/ipykernel_66434/1243303194.py:7:20
    %38 : int = aten::add(%37, %13) # /tmp/ipykernel_66434/1243303194.py:7:20
     = prim::Loop(%38, %33) # /tmp/ipykernel_66434/1243303194.py:7:4
      block0(%ja.1 : int):
        %43 : Tensor[] = prim::ListConstruct()
         = prim::Loop(%na.1, %33) # /tmp/ipykernel_66434/1243303194.py:8:37
          block0(%jb.1 : int):
            %49 : Tensor[] = prim::ListConstruct()
             = prim::Loop(%ma.1, %33) # /tmp/ipykernel_66434/1243303194.py:9:27
              block0(%ib.1 : int):
             

/awf-vmdisk-1/awf/dev/knossos-ksc/src/python/ksc/torch_frontend.py:61: UserWarning: knossos: unimplmented node kind: prim::ListUnpack
  warnings.warn(f"knossos: {msg}")
/awf-vmdisk-1/awf/dev/knossos-ksc/src/python/ksc/torch_frontend.py:61: UserWarning: knossos: unimplmented node kind: prim::ListUnpack
  warnings.warn(f"knossos: {msg}")


AssertionError: 